
---
<big><big><big><big><big><big>Sieci neuronowe 2018/19</big></big></big></big></big></big>

---
<big><big><big><big><big>Operacje na przestrzeni latent</big></big></big></big></big>

---

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

In [ ]:
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

plt.style.use("fivethirtyeight")

from bokeh.io import gridplot, output_file, show
from bokeh.plotting import figure, output_notebook
from bkcharts import Scatter

In [ ]:
output_notebook()

In [ ]:
sns.set(font_scale=2.0)

Image inclusion
<img src="../nn_figures/" width="100%">

# Operacje w przestrzeni latent

### kodowanie i dekodowanie danych wejściowych
* ze zbioru uczącego, ze zbioru testującego
* miarą jest wartość błędu odtwarzania

### samplowanie z okolic
* kodujemy przykład $x$
* samplujemy z okolic
  * enkoder zwraca __rozkład__ podany przez parametry
  * dla rozkładu normalnego samplujemy z tego rozkładu o wyliczonych parametrach $z=(\mu,\sigma^2)$ i dekodujemy (dekoder jest deterministyczny!)
* wygenerowane przykłady powinny należeć do przykładów z rozkładu $X$
* wygenerowane przykłady powinny być __podobne__ do oryginalnego
  * co to znaczy _podobne_?
  * podobne zdjęcia, częsty problem z dobrą oceną
  * przykłady mogą pochodzić ze zbiorów o małej wariancji danych, np. twarze (CelebA)
  * niezależne miary, np. biblioteki dla oceny grafów z danej klasy

### interpolacja
* interesuje nas przejście między dwoma przykładami
* kodujemy $x_1$ i $x_2$ dostając $z_1, z_2$
* interpolacja między nimi
* liniowa?
* paradoks "mydlanej bańki"
  * niech $P_Z$ $D$-wymiarowym rozkładem normalnym $\mathcal{N}(0,\mathbb{I})$
  * losując $z\sim P(Z)$ losujemy $D$ zmiennych $z_i\sim\mathcal{N}(0,1)$ _niezależnie_
  * kwadrat normy $Y=z_1^2+z_2^2+\ldots+z_D^2$
  * zmienna losowa $Y$ ma rozkład $\chi^2$
  * dla $D>30$ korzystamy w przybliżeniu z rozkładu normalnego $\mathcal{N}(\sqrt{2D-1},1)$
  * a więc normy wszystkich przykładów dla np. $D=100$ mają wartość oczekiwaną rzędu $14$!
  * oznacza to, że wszystkie $z\sim p(z)$ leżą na bańce o niewielkiej grubości
    * grubość jest stała, więc stosunkowo coraz węższa w stosunku do średnicy
  * punkty interpolacji liniowej przechodzą przez obszar o małej masie prawdopodobieństwa
* jak sobie z tym poradzić w interpolacjach?
  * interpolacje sferyczne
    * heureza oparta na paradoksie bańki
  * inny rozkład
    * weźmy model o ustalonym prior $p(z)$
    * losujemy dwa $z$, obliczamy punkt w środku $z=0.5z_1+0.5z_2$
    * jaki będzie rozkład punktów środkowych?
    * dla rozkładów prior o znanej średniej, rozkład środkowych będzie __inny__
    * uczenie i interpolacje będą na innych rozkładach, stąd interpolacje nie mogą być poprawne
    * potrzebny rozkład prior, dla którego średnia nie będzie się różnić
    * takim rozkładem jest np. rozkład Cauchy-ego
    $$C(x)=\frac{1}{\pi}\frac{\lambda}{\lambda^2+(x-\alpha)^2}$$
      * $\alpha$ to parametr położenia, $\lambda>0$ skali
    * to tzw. rozkład niewidomego łucznika
    * rozkład Cauchy-ego __nie ma__ średniej, wariancji, ani wyższych momentów
    * rozkład średnich $z\sim C$ jest identyczny z $C$
    * można nauczyć model biorąc $p(z)\sim C$
    * jednak $C$ ma "grube" ogony i samplowanie stamtąd daje nienajlepsze wyniki
    * inne rozwiązanie 
      * użyć normalny prior do uczenia
      * przy interpolacji
        * wylosować dwa punkty z normalnego
        * przez funkcje $CDF$ rozkładu normalnego i rozkład Cauchy przeprowadzić je do przestrzeni rozkładu Cauchy
        * przeprowadzić interpolację liniową
        * wykonać odwzorowanie odwrotne
      * możliwe, dające ten sam rezultat, rozwiązanie przenoszące rozkład punktów średnich do założonego rozkładu wykorzystując tzw. earth-moving (OT)
      
      
<img src="../nn_figures/automata-manifold.pdf" width="100%"> 